In [27]:
import pulp

In [28]:
model_ = pulp.LpProblem("Q1", pulp.LpMinimize)
products_ = ["A", "B", "C"]
Month_ = ["January", "February", "March", "April"]
x_var = pulp.LpVariable.dicts(f"Monthly_production", [(i,j) for i in products_ for j in Month_], lowBound=0, cat=pulp.LpInteger)
hour_cost ={"A":1.2, "B":1.3, "C":1}
holding_cost ={"A":1, "B":2, "C":0.5}
production_cost ={"A":20, "B":30, "C":10}
monthly_availiblity = {"January":500, "February":450, "March":400, "April":350}
# Monthly Available Time
for month_ in Month_:
    model_ += hour_cost["A"]*x_var["A", month_] + hour_cost["B"]*x_var["B", month_] +\
        hour_cost["C"]*x_var["C", month_] <= monthly_availiblity[month_], f"TimeAvailableIN{month_}"

# Invenory
JanuaryInventory = {"A":0, "B":0, "C":0}
FebruaryInventory = {"A":x_var["A", "January"] + JanuaryInventory["A"]-100,
                    "B":x_var["B", "January"] + JanuaryInventory["B"]-80, 
                    "C":x_var["C", "January"] + JanuaryInventory["C"]-120}
MarchInventory = {"A":x_var["A", "February"] + FebruaryInventory["A"]-110,
                    "B":x_var["B", "February"] + FebruaryInventory["B"]-90, 
                    "C":x_var["C", "February"] + FebruaryInventory["C"]-130}
AprilInventory = {"A":x_var["A", "March"] + MarchInventory["A"]-120,
                    "B":x_var["B", "March"] + MarchInventory["B"]-100, 
                    "C":x_var["C", "March"] + MarchInventory["C"]-140}
MayInventory = {"A":x_var["A", "April"] + AprilInventory["A"]-130,
                    "B":x_var["B", "April"] + AprilInventory["B"]-110, 
                    "C":x_var["C", "April"] + AprilInventory["C"]-150}
# Demand Constraint
Demand_ = {
    "January": {"A": 100, "B": 80, "C": 120},
    "February": {"A": 110, "B": 90, "C": 130},
    "March": {"A": 120, "B": 100, "C": 140},
    "April": {"A": 130, "B": 110, "C": 150},
}
# January 
for sku_ in products_:
    model_ += x_var[sku_, "January"] + JanuaryInventory[sku_] >= Demand_["January"][sku_], f"{sku_}DemandinJanuary"
# February
    model_ += x_var[sku_, "February"] + FebruaryInventory[sku_] >= Demand_["February"][sku_], f"{sku_}DemandinFebruary"
    model_ += x_var[sku_, "March"] + MarchInventory[sku_] >= Demand_["March"][sku_], f"{sku_}DemandinMarch" 
    model_ += x_var[sku_, "April"] + AprilInventory[sku_] >= Demand_["April"][sku_], f"{sku_}DemandinApril"
Production_overall_cost = production_cost["A"]*pulp.lpSum(x_var["A", month_] for month_ in Month_) +\
                            production_cost["B"]*pulp.lpSum(x_var["B", month_] for month_ in Month_) +\
                            production_cost["C"]*pulp.lpSum(x_var["C", month_] for month_ in Month_)

inventory_list = [JanuaryInventory, FebruaryInventory, MarchInventory, AprilInventory, MayInventory]
total_inventory = {}
for month_inventory in inventory_list:
    for key, value in month_inventory.items():
        total_inventory[key] = total_inventory.get(key, 0) + value

Holding_overall_cost = holding_cost["A"]*total_inventory["A"] + \
                        holding_cost["B"]*total_inventory["B"] + \
                        holding_cost["C"]*total_inventory["C"]

model_ += Holding_overall_cost + Production_overall_cost, "Objective"
model_.solve()
print("Model Status:{}".format(pulp.LpStatus[model_.status]))
for v in model_.variables():
	  print(v.name, "=", v.varValue, "\tReduced Cost =", v.dj)
print("Objective=", pulp.value(model_.objective))

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/yanhanjun/Library/Python/3.9/lib/python/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/8t/3xrvdszj1g75llc2jb40dnvw0000gn/T/6b01f29b67724709b4cf8a5f6e5a3558-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/8t/3xrvdszj1g75llc2jb40dnvw0000gn/T/6b01f29b67724709b4cf8a5f6e5a3558-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 21 COLUMNS
At line 100 RHS
At line 117 BOUNDS
At line 130 ENDATA
Problem MODEL has 16 rows, 12 columns and 42 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 29677 - 0.00 seconds
Cgl0004I processed model has 13 rows, 12 columns (12 integer (0 of which binary)) and 39 elements
Cutoff increment increased from 1e-05 to 0.4999
Cbc0012I Integer solution of 29677 found by DiveCoefficient after 0 iterations and 0 nodes (0.02 seconds)
C

In [29]:
# Sub-question B
model_ = pulp.LpProblem("Q1", pulp.LpMinimize)
products_ = ["A", "B", "C"]
Month_ = ["January", "February", "March", "April"]
x_var = pulp.LpVariable.dicts(f"Monthly_production", [(i,j) for i in products_ for j in Month_], lowBound=0, cat=pulp.LpInteger)
hour_cost ={"A":1.2, "B":1.3, "C":1}
holding_cost ={"A":1, "B":2, "C":0.5}
production_cost ={"A":20, "B":30, "C":10}
monthly_availiblity = {"January":500, "February":450, "March":400, "April":350}
# Monthly Available Time
for month_ in Month_:
    model_ += hour_cost["A"]*x_var["A", month_] + hour_cost["B"]*x_var["B", month_] +\
        hour_cost["C"]*x_var["C", month_] <= monthly_availiblity[month_], f"TimeAvailableIN{month_}"

# Invenory
JanuaryInventory = {"A":0, "B":0, "C":0}
FebruaryInventory = {"A":x_var["A", "January"] + JanuaryInventory["A"]-100,
                    "B":x_var["B", "January"] + JanuaryInventory["B"]-80, 
                    "C":x_var["C", "January"] + JanuaryInventory["C"]-120}
MarchInventory = {"A":x_var["A", "February"] + FebruaryInventory["A"]-110,
                    "B":x_var["B", "February"] + FebruaryInventory["B"]-90, 
                    "C":x_var["C", "February"] + FebruaryInventory["C"]-130}
AprilInventory = {"A":x_var["A", "March"] + MarchInventory["A"]-120,
                    "B":x_var["B", "March"] + MarchInventory["B"]-100, 
                    "C":x_var["C", "March"] + MarchInventory["C"]-140}
MayInventory = {"A":x_var["A", "April"] + AprilInventory["A"]-130,
                    "B":x_var["B", "April"] + AprilInventory["B"]-110, 
                    "C":x_var["C", "April"] + AprilInventory["C"]-150}
# Demand Constraint
Demand_ = {
    "January": {"A": 100, "B": 80, "C": 120},
    "February": {"A": 110, "B": 90, "C": 130},
    "March": {"A": 120, "B": 100, "C": 140},
    "April": {"A": 130, "B": 110, "C": 150},
}
# January 
for sku_ in products_:
    model_ += x_var[sku_, "January"] + JanuaryInventory[sku_] >= Demand_["January"][sku_], f"{sku_}DemandinJanuary"
# February
    model_ += x_var[sku_, "February"] + FebruaryInventory[sku_] >= Demand_["February"][sku_], f"{sku_}DemandinFebruary"
    model_ += x_var[sku_, "March"] + MarchInventory[sku_] >= Demand_["March"][sku_], f"{sku_}DemandinMarch" 
    model_ += x_var[sku_, "April"] + AprilInventory[sku_] >= Demand_["April"][sku_], f"{sku_}DemandinApril"

Exclued_Month = ["January", "February", "April"]
Production_overall_cost = production_cost["A"]*pulp.lpSum(x_var["A", month_] for month_ in Month_) +\
                            production_cost["B"]*pulp.lpSum(x_var["B", month_] for month_ in Exclued_Month) +\
                            (production_cost["B"]+5)*x_var["B", "March"] +\
                            production_cost["C"]*pulp.lpSum(x_var["C", month_] for month_ in Month_)

inventory_list = [JanuaryInventory, FebruaryInventory, MarchInventory, AprilInventory, MayInventory]
total_inventory = {}
for month_inventory in inventory_list:
    for key, value in month_inventory.items():
        total_inventory[key] = total_inventory.get(key, 0) + value

Holding_overall_cost = holding_cost["A"]*total_inventory["A"] + \
                        holding_cost["B"]*total_inventory["B"] + \
                        holding_cost["C"]*total_inventory["C"]

model_ += Holding_overall_cost + Production_overall_cost, "Objective"
model_.solve()
print("Model Status:{}".format(pulp.LpStatus[model_.status]))
for v in model_.variables():
	  print(v.name, "=", v.varValue, "\tReduced Cost =", v.dj)
print("Objective=", pulp.value(model_.objective))

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/yanhanjun/Library/Python/3.9/lib/python/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/8t/3xrvdszj1g75llc2jb40dnvw0000gn/T/22c00b3ae5584248b0f1d822ccb75360-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/8t/3xrvdszj1g75llc2jb40dnvw0000gn/T/22c00b3ae5584248b0f1d822ccb75360-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 21 COLUMNS
At line 100 RHS
At line 117 BOUNDS
At line 130 ENDATA
Problem MODEL has 16 rows, 12 columns and 42 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 29829 - 0.00 seconds
Cgl0004I processed model has 13 rows, 12 columns (12 integer (0 of which binary)) and 39 elements
Cutoff increment increased from 1e-05 to 0.4999
Cbc0012I Integer solution of 29829 found by DiveCoefficient after 0 iterations and 0 nodes (0.00 seconds)
C

In [30]:
# Sub-question C
model_ = pulp.LpProblem("Q1", pulp.LpMinimize)
products_ = ["A", "B", "C"]
Month_ = ["January", "February", "March", "April"]
x_var = pulp.LpVariable.dicts(f"Monthly_production", [(i,j) for i in products_ for j in Month_], lowBound=0, cat=pulp.LpInteger)
hour_cost ={"A":1.2, "B":1.3, "C":1}
holding_cost ={"A":1, "B":2, "C":0.5}
production_cost ={"A":20, "B":30, "C":10}
monthly_availiblity = {"January":500, "February":450, "March":400, "April":350}
# Monthly Available Time
for month_ in Month_:
    model_ += hour_cost["A"]*x_var["A", month_] + hour_cost["B"]*x_var["B", month_] +\
        hour_cost["C"]*x_var["C", month_] <= monthly_availiblity[month_], f"TimeAvailableIN{month_}"

# Invenory
JanuaryInventory = {"A":0, "B":0, "C":0}
FebruaryInventory = {"A":x_var["A", "January"] + JanuaryInventory["A"]-100,
                    "B":x_var["B", "January"] + JanuaryInventory["B"]-80, 
                    "C":x_var["C", "January"] + JanuaryInventory["C"]-120}
MarchInventory = {"A":x_var["A", "February"] + FebruaryInventory["A"]-110,
                    "B":x_var["B", "February"] + FebruaryInventory["B"]-90, 
                    "C":x_var["C", "February"] + FebruaryInventory["C"]-130}
AprilInventory = {"A":x_var["A", "March"] + MarchInventory["A"]-120,
                    "B":x_var["B", "March"] + MarchInventory["B"]-100, 
                    "C":x_var["C", "March"] + MarchInventory["C"]-140}
MayInventory = {"A":x_var["A", "April"] + AprilInventory["A"]-130,
                    "B":x_var["B", "April"] + AprilInventory["B"]-110, 
                    "C":x_var["C", "April"] + AprilInventory["C"]-150}
# Demand Constraint
Demand_ = {
    "January": {"A": 100, "B": 80, "C": 120},
    "February": {"A": 110, "B": 90, "C": 130},
    "March": {"A": 120, "B": 100, "C": 140},
    "April": {"A": 130, "B": 110, "C": 150},
}
# January 
for sku_ in products_:
    model_ += x_var[sku_, "January"] + JanuaryInventory[sku_] >= Demand_["January"][sku_], f"{sku_}DemandinJanuary"
# February
    model_ += x_var[sku_, "February"] + FebruaryInventory[sku_] >= Demand_["February"][sku_], f"{sku_}DemandinFebruary"
    model_ += x_var[sku_, "March"] + MarchInventory[sku_] >= Demand_["March"][sku_], f"{sku_}DemandinMarch" 
    model_ += x_var[sku_, "April"] + AprilInventory[sku_] >= Demand_["April"][sku_], f"{sku_}DemandinApril"
Production_overall_cost = production_cost["A"]*pulp.lpSum(x_var["A", month_] for month_ in Month_) +\
                            production_cost["B"]*pulp.lpSum(x_var["B", month_] for month_ in Month_) +\
                            production_cost["C"]*pulp.lpSum(x_var["C", month_] for month_ in Month_)
# Increased demand for C in April
model_ += x_var["C", "April"] + AprilInventory["C"] >= (Demand_["April"]["C"] + 20), f"New_{sku_}DemandinApril"
inventory_list = [JanuaryInventory, FebruaryInventory, MarchInventory, AprilInventory, MayInventory]
total_inventory = {}
for month_inventory in inventory_list:
    for key, value in month_inventory.items():
        total_inventory[key] = total_inventory.get(key, 0) + value

Holding_overall_cost = holding_cost["A"]*total_inventory["A"] + \
                        holding_cost["B"]*total_inventory["B"] + \
                        holding_cost["C"]*total_inventory["C"]

model_ += Holding_overall_cost + Production_overall_cost, "Objective"
model_.solve()
print("Model Status:{}".format(pulp.LpStatus[model_.status]))
for v in model_.variables():
	  print(v.name, "=", v.varValue, "\tReduced Cost =", v.dj)
print("Objective=", pulp.value(model_.objective))

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/yanhanjun/Library/Python/3.9/lib/python/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/8t/3xrvdszj1g75llc2jb40dnvw0000gn/T/e69d4e4b1a9743e5804567bd09a5b1bd-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/8t/3xrvdszj1g75llc2jb40dnvw0000gn/T/e69d4e4b1a9743e5804567bd09a5b1bd-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 22 COLUMNS
At line 105 RHS
At line 123 BOUNDS
At line 136 ENDATA
Problem MODEL has 17 rows, 12 columns and 46 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 29917 - 0.00 seconds
Cgl0004I processed model has 13 rows, 12 columns (12 integer (0 of which binary)) and 39 elements
Cutoff increment increased from 1e-05 to 0.4999
Cbc0012I Integer solution of 29917 found by DiveCoefficient after 0 iterations and 0 nodes (0.00 seconds)
C